In [2]:
from dotenv import load_dotenv

load_dotenv("../.envrc")

True

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

Chroma().delete_collection()

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

[Document(id='692435c2-000d-430b-911a-6e7f5c59d51c', metadata={'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-04-01T21:26:24+09:00', 'page': 4, 'total_pages': 14, 'page_label': '5', 'moddate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='4e759668-ac32-4dc5-aabc-96fd1611412b', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'total_pages': 14,

In [6]:
result_score = db.similarity_search_with_score("국회의원의 의무")
result_r_score = db.similarity_search_with_relevance_scores("국회의원의 의무")

print(result_score)
print(result_r_score)

print("[유사 청크 1순위]")
print(result_score[0][0].page_content)

print("\n\n[점수]")
print(result_score[0][1], result_score[1][1]) # 낮을 수록 유사도 높음
print(result_r_score[0][1], result_r_score[1][1]) # 높을 수록 유사도 높음

[(Document(id='692435c2-000d-430b-911a-6e7f5c59d51c', metadata={'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'page': 4, 'total_pages': 14}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'), 0.800208330154419), (Document(id='4e759668-ac32-4dc5-aabc-96fd1611412b', metadata={'moddate': '2024-04-01T21:26:24+09:00', 'page_label': '5', 'total_pages': 14, 'cre

In [7]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 1})
retriever.invoke("국회의원의 의무")

[Document(id='692435c2-000d-430b-911a-6e7f5c59d51c', metadata={'total_pages': 14, 'page': 4, 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'creationdate': '2024-04-01T21:26:24+09:00'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.')]

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [13]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
docs = retriever.invoke("국회의원의 의무")
for doc in docs:
    print(doc.page_content)
    print("--" * 20)

③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.
----------------------------------------
법제처                                                            12                                                       국가법령정보센터
대한민국헌법
④위원은 정당에 가입하거나 정치에 관여할 수 없다.
⑤위원은 탄핵 또는 금고 이상의 형의 선고에 의하지 아니하고는 파면되지 아니한다.
⑥중앙선거관리위원회는 법령의 범위 안에서 선거관리ㆍ국민투표관리 또는 정당사무에 관한 규칙을 제정할 수 있
으며, 법률에 저촉되지 아니하는 범위 안에서 내부규율에 관한 규칙을 제정할 수 있다.
⑦각급 선거관리위원회의 조직ㆍ직무범위 기타 필요한 사항은 법률로 정한다.
 
제115조 ①각급 선거관리위원회는 선거인명부의 작성 등 선거사무와 국민투표사무에 관하여 관계 행정기관에 필요한
지시를 할 수 있다.
②제1항의 지시를 받은 당해 행정기관은 이에 응하여야 

In [14]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
docs = retriever.invoke("국회의원의 의무")
for doc in docs:
    print(doc.page_content)
    print("--" * 20)

③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.
----------------------------------------
③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필

In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [17]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-4o-mini",
                temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['국회의원이 수행해야 하는 의무는 어떤 것들이 있나요?  ', '국회의원의 책임과 의무는 무엇인지 알고 싶습니다.  ', '국회의원으로서의 주요 의무는 무엇인가요?']


[Document(id='95fd28ef-cb98-4254-93e9-90309a352140', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'page': 5, 'total_pages': 14, 'page_label': '6', 'creator': 'PyPDF'}, page_content='②국회는 의원의 자격을 심사하며, 의원을 징계할 수 있다.\n③의원을 제명하려면 국회재적의원 3분의 2 이상의 찬성이 있어야 한다.\n④제2항과 제3항의 처분에 대하여는 법원에 제소할 수 없다.\n \n제65조 ①대통령ㆍ국무총리ㆍ국무위원ㆍ행정각부의 장ㆍ헌법재판소 재판관ㆍ법관ㆍ중앙선거관리위원회 위원ㆍ감사\n원장ㆍ감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의\n소추를 의결할 수 있다.\n②제1항의 탄핵소추는 국회재적의원 3분의 1 이상의 발의가 있어야 하며, 그 의결은 국회재적의원 과반수의 찬성이\n있어야 한다. 다만, 대통령에 대한 탄핵소추는 국회재적의원 과반수의 발의와 국회재적의원 3분의 2 이상의 찬성이'),
 Document(id='88cae39b-3c8f-4c8c-878c-b7e8448e4b0a', metadata={'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page_label': '6', 'page': 5, 'moddate': '2024-04-01T21:26:24+09:00', 'total_pages': 14, 'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'producer': 'iText 2.1.7 by 

In [18]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf"),
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# 상위 문서 저장 위한 레이어 선언
store = InMemoryByteStore()
id_key = "doc_id"
# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

#문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [21]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)
#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [22]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

[하위 청크] 

③공공필요에 의한 재산권의 수용ㆍ사용 또는 제한 및 그에 대한 보상은 법률로써 하되, 정당한 보상을 지급하여야
한다.
 
제24조 모든 국민은 법률이 정하는 바에 의하여 선거권을 가진다.
 
제25조 모든 국민은 법률이 정하는 바에 의하여 공무담임권을 가진다.
 
제26조 ①모든 국민은 법률이 정하는 바에 의하여 국가기관에 문서로 청원할 권리를 가진다.
②국가는 청원에 대하여 심사할 의무를 진다.
 
제27조 ①모든 국민은 헌법과 법률이 정한 법관에 의하여 법률에 의한 재판을 받을 권리를 가진다.
②군인 또는 군무원이 아닌 국민은 대한민국의 영역 안에서는 중대한 군사상 기밀ㆍ초병ㆍ초소ㆍ유독음식물공급
ㆍ포로ㆍ군용물에 관한 죄중 법률이 정한 경우와 비상계엄이 선포된 경우를 제외하고는 군사법원의 재판을 받지
--------------------------------------------------
[상위 청크] 

법제처                                                            3                                                       국가법령정보센터
대한민국헌법
제21조 ①모든 국민은 언론ㆍ출판의 자유와 집회ㆍ결사의 자유를 가진다.
②언론ㆍ출판에 대한 허가나 검열과 집회ㆍ결사에 대한 허가는 인정되지 아니한다.
③통신ㆍ방송의 시설기준과 신문의 기능을 보장하기 위하여 필요한 사항은 법률로 정한다.
④언론ㆍ출판은 타인의 명예나 권리 또는 공중도덕이나 사회윤리를 침해하여서는 아니된다. 언론ㆍ출판이 타인의
명예나 권리를 침해한 때에는 피해자는 이에 대한 피해의 배상을 청구할 수 있다.
 
제22조 ①모든 국민은 학문과 예술의 자유를 가진다.
②저작자ㆍ발명가ㆍ과학기술자와 예술가의 권리는 법률로써 보호한다.
 
제23조 ①모든 국민의 재산권은 보장된다. 그 내용과 한계는 법률로 정한다.
②재산권의 행사는 공공복리에 적합하도록 하여야 한다.
③공공필요에 의한 재산권의 수용

In [23]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_chroma import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

Chroma().delete_collection()

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
docs

[Document(id='015a9b71-7616-4eb4-a9d0-394c68fbcc85', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='c6042b69-e1df-40a1-bd71-0c3101f97858', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='89d49725-937f-46e3-9af7-994d3d1e8b53', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='f60c33b9-9ccb-45cf-8523-0b24d73ba786', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='2896f67f-5b47-4738-bcf5-4ac4222c29c8', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='6a64daa0-b2a0-4d3a-9d04-fd518b869efe', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='0f5c775f-8662-403d-ab97-d585253cfbd6', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='6e3ce86b-56cb-4ebe-b6ff-fd3550790f13', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='1c34b317-e9ef-4043-a02b-16d67c4ddbf7', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.')]

In [24]:
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(id='015a9b71-7616-4eb4-a9d0-394c68fbcc85', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='89d49725-937f-46e3-9af7-994d3d1e8b53', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='2896f67f-5b47-4738-bcf5-4ac4222c29c8', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='0f5c775f-8662-403d-ab97-d585253cfbd6', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='1c34b317-e9ef-4043-a02b-16d67c4ddbf7', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='6e3ce86b-56cb-4ebe-b6ff-fd3550790f13', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='6a64daa0-b2a0-4d3a-9d04-fd518b869efe', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='f60c33b9-9ccb-45cf-8523-0b24d73ba786', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='c6042b69-e1df-40a1-bd71-0c3101f97858', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.')]